# Interactive choropleth maps with Python and Folium for Srilanka (Grama Niladari Level)

In [9]:
#Let's import libraries we need
import pandas as pd
import geopandas as gpd
import folium

In [10]:
# importing geo json files 
gng=gpd.read_file('Grama_geo.json')

In [11]:
# loading first two rows of the data set
gng.head(2)

,ADM4_EN,ADM4_SI,ADM4_TA,ADM4_PCODE,ADM4_REF,ADM4ALT1EN,ADM4ALT2EN,ADM4ALT1SI,ADM4ALT2SI,ADM4ALT1TA,...,ADM0_EN,ADM0_SI,ADM0_TA,ADM0_PCODE,Shape_Leng,Shape_Area,date,validOn,validTo,geometry
0,04 1/2 Kanuwa,4 ½ කණුව,,LK7142090,,,,,,,...,Sri Lanka,ශ්‍රී ලංකා,இலங்கை,LK,0.141404,0.00049,2019-03-04T00:00:00,2020-03-05T00:00:00,1899-11-30T00:00:00,"POLYGON ((80.23123 8.18024, 80.23082 8.18030, ..."
1,07 Gammanaya,7 වන ගම්මානය,,LK6224040,,,,,,,...,Sri Lanka,ශ්‍රී ලංකා,இலங்கை,LK,0.122450,0.00034,2019-03-04T00:00:00,2020-03-05T00:00:00,1899-11-30T00:00:00,"POLYGON ((80.02773 7.95962, 80.02773 7.95962, ..."


In [12]:
#removing other data except "ADM4_PCODE" and "geometry"
gng=gng[['ADM4_PCODE','geometry']]

In [13]:
# loading aditional data
gn_a=pd.read_csv('grama_niladari_aditional_data.csv')
gn_a.rename(columns={'admin4Pcode':'ADM4_PCODE'}, inplace=True)
gn_a.head()

,admin4Name_en,admin4Name_si,ADM4_PCODE,risk_level,patients,population
0,04 1/2 Kanuwa,4 ½ කණුව,LK7142090,1.0,5.0,60.0
1,07 Gammanaya,7 වන ගම්මානය,LK6224040,2.0,10.0,120.0
2,08 Ela,අටේ ඇල,LK2121105,3.0,15.0,180.0
3,18A. Janapadaya,18 A ජනපදය,LK5242140,4.0,20.0,240.0
4,2 Ela - Kagama,කාගම ඇල 2,LK7160115,5.0,25.0,300.0


In [14]:
grama=gng.merge(gn_a,on="ADM4_PCODE")
grama.head()

,ADM4_PCODE,geometry,admin4Name_en,admin4Name_si,risk_level,patients,population
0,LK7142090,"POLYGON ((80.23123 8.18024, 80.23082 8.18030, ...",04 1/2 Kanuwa,4 ½ කණුව,1.000000,5.000000,60.00000
1,LK6224040,"POLYGON ((80.02773 7.95962, 80.02773 7.95962, ...",07 Gammanaya,7 වන ගම්මානය,2.000000,10.000000,120.00000
2,LK2121105,"POLYGON ((80.96122 7.38586, 80.96133 7.38591, ...",08 Ela,අටේ ඇල,3.000000,15.000000,180.00000
3,LK5242140,"POLYGON ((81.73889 7.16753, 81.73871 7.16744, ...",18A. Janapadaya,18 A ජනපදය,4.000000,20.000000,240.00000
4,LK4100000,"POLYGON ((80.25787 9.62729, 80.25784 9.62726, ...",[unknown],[unknown],376.868743,1884.343715,22612.12458


In [15]:
# Defining the style function and Highlight function 

style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}



# declaring the map with starting point
m = folium.Map(location=[7.8731, 80.7718], zoom_start=8)

choropleth=folium.Choropleth(
    geo_data=grama,
    name='choropleth',
    data= grama,
    columns=['ADM4_PCODE', 'risk_level'],
    key_on='properties.ADM4_PCODE',
    fill_color='RdPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    
    legend_name='Covid-19 risk Level with dummy data',

).add_to(m)

# adding some geo loacation (location pins )

folium.Marker(
    location=[6.905655, 79.927357],
    popup='Point one',
    
    icon=folium.Icon(icon='cloud')
).add_to(m)

folium.Marker(
    location=[6.7106,79.9074],
    popup='Point two',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

folium.Marker(
    location=[7.9403,81.01886823],
    popup='Point three',
    icon=folium.Icon(color='blue', icon='bar-chart')
).add_to(m)

folium.Marker(
    location=[9.3803, 80.3770],
    popup='Point four',
    icon=folium.Icon(color='red', icon='university')
).add_to(m)

#adding the same geo layer again on top of previouse layer to make interactivity 

INT = folium.features.GeoJson(
    grama,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['admin4Name_en','patients','population'],
        aliases=['GN name: ','Numer of Patients:','Population'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
m.add_child(INT)
m.keep_in_front(INT)
folium.LayerControl().add_to(m)


In [16]:
m.save('grama_niladari.html')

